In [2]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
from tqdm.autonotebook import tqdm
#import pandas as pd
import mysql.connector
import codecs
codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)
import re
import conllu

In [3]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  database='pruned_commits_db2',
  charset='utf8mb4')

cur = db.cursor()
cur.execute('SELECT lemma FROM lemmas ORDER BY id LIMIT 100')
top_lemmas = set([i[0] for i in cur.fetchall()])

In [4]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  database='small_commits_db',
  charset='utf8mb4')

cur = db.cursor()

In [28]:
def yield_pos_short():
    #mycursor.execute('SELECT conllu, language FROM C LEFT JOIN R ON C.repo_id=R.gh_id LIMIT 1000;')
    cur.execute('SELECT conllu, date FROM C LIMIT 100000')
    text = cur.fetchone()
    while text:
        #doc = conllu.parse(text[0])
        doc = re.findall('sent_id', text[0])
        date = text[1]
        if len(doc) > 0:
            doc = conllu.parse(text[0])
            if len(doc[0].tokens) <= 20:
                string = []
                if doc[0].tokens[0]['lemma'] in top_lemmas:
                    string.append(doc[0].tokens[0]['lemma'])
                    for i in doc[0].tokens[1:]:
                        string.append(i['upostag'])
                    yield ' '.join(string)+'\t'+date[:4]
        text = cur.fetchone()

In [29]:
file = open('/home/dkbrz/git_commits/top_first_freq.txt', 'w')
for i in tqdm(yield_pos_short()):
    print (i, file=file)

In [30]:
def yield_pos_short():
    #mycursor.execute('SELECT conllu, language FROM C LEFT JOIN R ON C.repo_id=R.gh_id LIMIT 1000;')
    cur.execute('SELECT date FROM C LIMIT 100000')
    text = cur.fetchone()
    while text:
        yield text[0][:4]
        text = cur.fetchone()
before = 0
after = 0
for date in tqdm(yield_pos_short()):
    #print(date)
    if date > '2016-01-01':
        after += 1
    elif date < '2010-01-01':
        before += 1 

In [31]:
before, after

(53200, 4870)

In [32]:
! sort /home/dkbrz/git_commits/top_first_freq.txt -T . > /home/dkbrz/git_commits/top_first_freq_sorted.txt

In [33]:
! tail /home/dkbrz/git_commits/top_first_freq_sorted.txt

work PART VERB NUM NOUN	2007
work PART VERB NUM NOUN PUNCT	2009
work PROPN PUNCT PROPN	2010
work PROPN VERB ADP NOUN NOUN	2007
work PUNCT ADV ADP ADP NUM NOUN PUNCT VERB ADP NOUN PART NOUN SCONJ PRON AUX ADP SYM NUM PUNCT	2016
work PUNCT NOUN PUNCT NOUN AUX ADJ PUNCT NOUN NOUN NOUN PUNCT	2009
work SCONJ VERB ADJ ADP DET NOUN	2009
work SCONJ VERB ADP ADJ NOUN VERB NOUN PUNCT	2008
work SCONJ VERB DET NOUN NOUN ADP NOUN NOUN	2008
work SCONJ VERB NOUN	2007


In [34]:
top = 10
k = 0
string = ''
counts = {}
before = 0
after = 0
for line in open('/home/dkbrz/git_commits/top_first_freq_sorted.txt'):
    text, date = line.strip().split('\t')
    if text != string:
        if before + after > top:
            counts[string] = [before, after]
        k = 1
        before = 0
        after = 0
        if date > '2016-01-01':
            after += 1
        elif date < '2010-01-01':
            before += 1
        string = text
    else:
        k += 1
        if date > '2016-01-01':
            after += 1
        elif date < '2010-01-01':
            before += 1

In [35]:
counts

{'add ADJ NOUN': [80, 6],
 'add ADJ NOUN ADP NOUN': [23, 4],
 'add ADJ NOUN ADP NOUN NOUN': [16, 4],
 'add ADJ NOUN ADP NOUN NOUN PUNCT': [11, 0],
 'add ADJ NOUN ADP NOUN PUNCT': [14, 0],
 'add ADJ NOUN ADP PROPN': [9, 3],
 'add ADJ NOUN ADP PROPN PUNCT': [12, 0],
 'add ADJ NOUN NOUN': [57, 2],
 'add ADJ NOUN NOUN NOUN': [12, 2],
 'add ADJ NOUN NOUN NOUN PUNCT': [10, 1],
 'add ADJ NOUN NOUN PUNCT': [26, 6],
 'add ADJ NOUN PUNCT': [56, 1],
 'add DET ADJ NOUN': [12, 0],
 'add DET ADJ NOUN PUNCT': [14, 0],
 'add DET NOUN': [17, 0],
 'add DET NOUN ADP NOUN': [11, 1],
 'add DET NOUN NOUN': [16, 2],
 'add DET NOUN NOUN PUNCT': [13, 0],
 'add NOUN': [148, 3],
 'add NOUN ADP ADJ NOUN': [39, 5],
 'add NOUN ADP ADJ NOUN NOUN': [20, 0],
 'add NOUN ADP ADJ NOUN PUNCT': [22, 0],
 'add NOUN ADP NOUN': [125, 23],
 'add NOUN ADP NOUN CCONJ NOUN': [12, 1],
 'add NOUN ADP NOUN NOUN': [64, 6],
 'add NOUN ADP NOUN NOUN NOUN': [11, 3],
 'add NOUN ADP NOUN NOUN PUNCT': [29, 2],
 'add NOUN ADP NOUN PUNCT': [

In [36]:
file = open('/home/dkbrz/git_commits/top_schemes.txt', 'w')
for key in sorted(counts, key=lambda x: sum(counts[x]), reverse=True):
    print('{}\t{}\t{}'.format(counts[key][0], counts[key][1], key), file=file)
file.close()

In [37]:
! head /home/dkbrz/git_commits/top_schemes.txt

579	3	merge NOUN PUNCT NOUN PUNCT ADP NOUN
520	1	merge NOUN PUNCT NOUN PUNCT
397	44	update NOUN
235	0	git PUNCT NOUN PUNCT NOUN PUNCT NUM NUM
192	12	version NOUN ADP NUM
188	10	add NOUN NOUN
181	2	merge NOUN PUNCT NOUN PUNCT NOUN PUNCT
141	22	remove ADJ NOUN
152	4	merge NOUN PUNCT NOUN PUNCT ADP PUNCT NOUN
148	3	add NOUN


In [38]:
before, after = 0, 0 
for key in counts:
    before += counts[key][0]
    after += counts[key][1]

In [39]:
before, after

(7659, 1026)

In [40]:
b, a = 53200, 4870
before/b, after/a

(0.14396616541353383, 0.2106776180698152)